In [1]:
from bs4 import BeautifulSoup
import os
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import re
import matplotlib.pyplot as plt
import config

In [ ]:
lst = os.listdir(config.data)
lst.sort()

In [ ]:
def parse_motif_page_correct(soup):
    list_of_regions = []
    list_of_nations = []
    list_of_texts = []
    list_of_codenames = []
    list_of_motifs = []
    
    try:
        code_name = soup.findAll("p", {"class": "NormalLin"})[0].get_text()
    except AttributeError:
        code_name = '-'
    except IndexError:
        code_name = '-'
        
    try:
        motif = soup.findAll("p", {"class": "NormalLis"})[0].get_text()
    except AttributeError:
        motif = '-'
    except IndexError:
        motif = '-'

    for region in soup.findAll("p", {"class": "NormalMai"}):
        try:
            reg = region.find("b").get_text()
            for nation in region.findAll("u"):
                list_of_regions.append(reg)
                list_of_nations.append(nation.get_text())
                list_of_texts.append(region)
                list_of_codenames.append(code_name)
                list_of_motifs.append(motif)
        except AttributeError:
#             print('except in find B: {}'.format(code_name))
            pass
            
    myth_df = pd.DataFrame(data={'region': list_of_regions, 
                                 'nation':list_of_nations, 
                                 'text': list_of_texts,
                                 'code_name':list_of_codenames, 
                                 'motif': list_of_motifs})
            
    return myth_df

In [ ]:
final_df = pd.DataFrame()
for file in tqdm(lst, total = len(lst)):
    with open('../data/myths/{}'.format(file), 'rb') as html:
        soup = BeautifulSoup(html, 'html.parser')
        tmp_df = parse_motif_page_correct(soup)
        final_df = pd.concat([final_df, tmp_df])
        
final_df.nation.value_counts()[:10]

достаем код

In [ ]:
final_df['code_true'] = final_df['code_name'].apply(lambda x: x.split(' ')[0].replace('.', ''))

местами неправильная кодировка, делаю правильную

In [ ]:
'РњРµР·РѕР°РјРµСЂРёРєР°.'.encode("windows-1251").decode("utf-8")

In [ ]:
def encode_only_decoded(x):
    try:
        return x.encode("windows-1251").decode("utf-8")
    except:
        return x

In [ ]:
final_df['region'] = final_df['region'].progress_apply(lambda x: encode_only_decoded(x))
final_df['nation'] = final_df['nation'].progress_apply(lambda x: encode_only_decoded(x))
final_df['text'] = final_df['text'].progress_apply(lambda x: encode_only_decoded(x))
final_df['code_name'] = final_df['code_name'].progress_apply(lambda x: encode_only_decoded(x))
final_df['motif'] = final_df['motif'].progress_apply(lambda x: encode_only_decoded(x))

нормальный code name

In [ ]:
def clean_code_name(x):
    return re.sub(r'[^а-яА-Я ]', '', x)

In [ ]:
final_df['code_name'] = final_df['code_name'].apply(clean_code_name)

нормальный текст: изначально в колонке - весь текст по региону, берем только то что после упоминания того народа, который у нас в nation, но до следующего \<u\> - подразумевается что там следующий народ

In [ ]:
def get_text(x):
    try:
        return str(x['text']).split('<u>{}</u>'.format(x['nation']))[1].split('<u>')[0]
    except IndexError:
        return '-'

In [ ]:
final_df['text'] = final_df.apply(lambda x: get_text(x), axis=1)

region+nation

In [ ]:
final_df['region'] = final_df['region'].apply(lambda x: re.sub(r'[^а-яА-Я ]', '', x).lower())
final_df['nation'] = final_df['nation'].apply(lambda x: re.sub(r'[^а-яА-Я ]', '', x).lower())

#например
final_df[final_df.code_true == 'D5'].head(20)# допускаю что ничего ценного в дубликатах текстов нет
final_df = final_df.drop_duplicates('text')

препроцессить тексты

In [ ]:
# допускаю что ничего ценного в дубликатах текстов нет
final_df = final_df.drop_duplicates('text')

from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

from nltk.corpus import stopwords
from pymystem3 import Mystem
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")
from string import punctuation

def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text

def preprocess(row):
    content = re.sub("[^А-Яа-я']+", ' ', str(row)).lower()
    content = list(preprocess_text(x) for x in content.split(' '))
    content = [x for x in content if x != '']
    return content

In [ ]:
final_df['preprocessed'] = final_df['text'].progress_apply(preprocess)

In [ ]:
final_df.iloc[43].preprocessed

In [ ]:
final_df.to_csv(config.final_df, index=False)